In [ ]:
from stable_baselines3 import SAC
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines3.common.env_util import make_vec_env

import gymnasium as gym
from gymnasium.wrappers import RescaleAction
import nav2d       
import numpy as np
import os
from tqdm import tqdm

test a given model:

In [ ]:
import pyautogui
    
# paths:
results_path = "results/Nav2D_SAC_SB3_results"
desired_result_path = "result_023"
run = "run_100"

# load:
model = SAC.load(os.path.join(results_path, desired_result_path, run))

# scales:
reward_scale = {
            "rew_head_scale" : 2.5,
            "rew_head_approach_scale" : 50.0,
            "rew_dist_scale" : 250.0,
            "rew_goal_scale" : 5000.0,
            "rew_obst_scale" : -1000.0
        }

# visualize:
testing_length = 50

# render settings:  e
width = 1280
height = 1280
default_camera_config = {"azimuth" : 90.0, "elevation" : -90.0, "distance" : 3, "lookat" : [0.0, 0.0, 0.0]}
camera_id = 2

DEFAULT_CAMERA = "overhead_camera"
ENABLE_FRAME = True
RENDER_EVERY_FRAME = True

# make a single environment:
env = gym.make("Nav2D-v0",
            render_mode = "human",
            width = width, 
            height = height,
            default_camera_config = default_camera_config, 
            camera_id = camera_id, 
            max_episode_steps = 1000, 
            is_eval = False,
            reward_scale_options = reward_scale)

# if DEFAULT_CAMERA=="overhead_camera": pyautogui.press('tab')
# if ENABLE_FRAME: pyautogui.press('e') 
# if not RENDER_EVERY_FRAME: pyautogui.press('d') 

# for every test episode:
for eps in range(testing_length):
    obs, _ = env.reset()
    done = False

    # while not done:
    while not done:
        action, _ = model.predict(obs, deterministic = True)
        nobs, reward, term, trunc, _ = env.step(action)
        done = term or trunc

        # advance observation, reset if not:
        obs = nobs if not done else env.reset()
    
        # render for user:
        env.render()
 
# close when done:
env.close()